In [24]:

from nes_py.wrappers import JoypadSpace
import gym_super_mario_bros
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT


In [29]:
#借助包gym_super_mario_bros创建
env = gym_super_mario_bros.make('SuperMarioBros-v0')
env = JoypadSpace(env, SIMPLE_MOVEMENT)

In [14]:
print("env.action_space: ", env.action_space)
# 0 是不动
# 1 是右
# 2 是右上
# 3 是向前走
# 4 是不动
# 5 是小跳
# 6 是后退

env.action_space:  Discrete(7)


下面这一步在最新版本的gym中可能会出现报错，问题在于env.step的返回值个数在不同的版本中不同。介于还在学习过程，建议先将gym的版本拉低

In [81]:

done = True
import cv2  
for step in range(7000):
    # if step % 100 == 0:
    #     print(step//100)
    # print(step)
    if done:
        state = env.reset()
    action = env.action_space.sample()
    state, reward, done, info = env.step(step //1000)
    print(state.shape)
    cv2.imshow("game",state)

    if cv2.waitKey(1) &  0xff == ord("q"): # 在英文状态下，按下按s键 q 会关闭显示窗口    
        break
cv2.destroyAllWindows()
env.close()
env = gym_super_mario_bros.make('SuperMarioBros-v0')
env = JoypadSpace(env, SIMPLE_MOVEMENT)

(240, 256, 3)
(240, 256, 3)
(240, 256, 3)
(240, 256, 3)
(240, 256, 3)
(240, 256, 3)
(240, 256, 3)
(240, 256, 3)
(240, 256, 3)
(240, 256, 3)
(240, 256, 3)
(240, 256, 3)
(240, 256, 3)
(240, 256, 3)
(240, 256, 3)
(240, 256, 3)
(240, 256, 3)
(240, 256, 3)
(240, 256, 3)
(240, 256, 3)
(240, 256, 3)
(240, 256, 3)
(240, 256, 3)
(240, 256, 3)
(240, 256, 3)
(240, 256, 3)
(240, 256, 3)
(240, 256, 3)
(240, 256, 3)
(240, 256, 3)
(240, 256, 3)
(240, 256, 3)
(240, 256, 3)
(240, 256, 3)
(240, 256, 3)
(240, 256, 3)
(240, 256, 3)
(240, 256, 3)
(240, 256, 3)
(240, 256, 3)
(240, 256, 3)
(240, 256, 3)
(240, 256, 3)
(240, 256, 3)
(240, 256, 3)
(240, 256, 3)
(240, 256, 3)
(240, 256, 3)
(240, 256, 3)
(240, 256, 3)
(240, 256, 3)
(240, 256, 3)
(240, 256, 3)
(240, 256, 3)
(240, 256, 3)
(240, 256, 3)
(240, 256, 3)
(240, 256, 3)
(240, 256, 3)
(240, 256, 3)
(240, 256, 3)
(240, 256, 3)
(240, 256, 3)
(240, 256, 3)
(240, 256, 3)
(240, 256, 3)
(240, 256, 3)
(240, 256, 3)
(240, 256, 3)
(240, 256, 3)
(240, 256, 3)
(240, 

KeyboardInterrupt: 

强化学习执行step动作的返回一般是标准的，所以这里的返回同前面的倒立摆，动作执行后，环境返回四个变量（obj:新的观测值（一帧rgb图片）、reward：执行该动作获得的奖励值（在游戏中的得分）、done:回合是否结束（你控制的小人是不是死了，对应回合结束）、info:额外信息（比如'life': 2，剩余2条命等））

In [2]:
import gym_super_mario_bros
from gym.spaces import Box
from gym import Wrapper
from nes_py.wrappers import JoypadSpace#BinarySpaceToDiscreteSpaceEnv
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT, COMPLEX_MOVEMENT, RIGHT_ONLY
import cv2
import numpy as np
import subprocess as sp

In [9]:
def process_frame(frame):
    if frame is not None:
        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY) #图像转换
        
        frame = cv2.resize(frame, (84, 84))[None, :, :] / 255. #裁剪合适大小，并归一化

        return frame
    else:
        return np.zeros((1,84, 84))

In [10]:

class CustomSkipFrame(Wrapper):
    def __init__(self, env, skip=8):
        super(CustomSkipFrame, self).__init__(env)
        self.observation_space = Box(low=0, high=255, shape=(4, 84, 84))
        self.skip = skip

    def step(self, action):
        total_reward = 0
        states = []
        state, reward, done, info = self.env.step(action)
        for i in range(self.skip):
            if not done:
                state, reward, done, info = self.env.step(action)
                total_reward += reward
                states.append(state)
            else:
                states.append(state)
        states = np.concatenate(states, 0)[None, :, :, :]
        return states.astype(np.float32), reward, done, info

    def reset(self):
        state = self.env.reset()
        states = np.concatenate([state for _ in range(self.skip)], 0)[None, :, :, :]
        return states.astype(np.float32)



In [12]:
class CustomReward(Wrapper):
    def __init__(self, env=None):
        super(CustomReward, self).__init__(env)
        self.observation_space = Box(low=0, high=255, shape=(1, 84, 84))
        self.curr_score = 0
        self.pre_x = 38 
    def step(self, action):
        state, reward, done, info = self.env.step(action) 
        state = process_frame(state)
        reward += (info["score"] - self.curr_score) / 20.
        reward -= 5 if info["x_pos"] - self.pre_x else -5 # 当保持原位时扣分，向前动就加分
        self.pre_x = info["x_pos"]
        if done  : reward -= 50 
        self.curr_score = info["score"]
        if done:
            if info["flag_get"]:
                reward += 500
            else:
                reward -= 500
        return state, reward/10. , done, info

    def reset(self):
        self.curr_score = 0
        return process_frame(self.env.reset())

In [13]:
#至此，我们完成了超级玛丽环境的自定义，封装如下：
def create_train_env(world, stage, action_type, output_path=None):
    env = gym_super_mario_bros.make("SuperMarioBros-{}-{}-v0".format(world, stage))
    if action_type == "right":
        actions = RIGHT_ONLY
    elif action_type == "simple":
        actions = SIMPLE_MOVEMENT
    else:
        actions = COMPLEX_MOVEMENT
    env = JoypadSpace(env, actions)
    env = CustomReward(env)
    env = CustomSkipFrame(env)
    return env, env.observation_space.shape[0], len(actions)

In [14]:
#测试一下
custom_env,_,_ = create_train_env(1,1,'simple')
print(custom_env)


<CustomSkipFrame<CustomReward<JoypadSpace<TimeLimit<SuperMarioBrosEnv<SuperMarioBros-1-1-v0>>>>>>


In [15]:
custom_env.reset()
done = True
flag =False 
t = 0 
import cv2  
for step in range(50000):
    if done == True :
        custom_env.reset()
        if flag:
            print(f"成功了,距离上一次成功经过了{step-t}步")
            t = step 



    action = custom_env.action_space.sample()
    states, reward, done, info = custom_env.step(action)
    ############# 显示过程 ##################
    # flag = info["flag_get"]
    # states = states[0]
    # for state in states:
    #     state.reshape(state.shape+(1,))
    #     cv2.imshow("test",state)
    # if cv2.waitKey(1) &  0xff == ord("q"): # 在英文状态下，按下按s键 q 会关闭显示窗口    
    #     break
    ##################################################
    
# cv2.destroyAllWindows()
custom_env.close()
cv2.destroyAllWindows()

发现一个问题在于如果不进行跳过帧这一操作时，马里奥有可能会出现一直小跳而跳不出较高的柱子的情况

教程中使用的是4帧的跳过，依然存在连续小跳的情况，于是将skip调整至8进行尝试。连续小跳的情况得到了较好的解决，但是存在马里奥往坑里跳的情况。